In [14]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/blend2/blend-2/1866.csv
/kaggle/input/blend2/blend-2/1872.csv
/kaggle/input/blend2/blend-2/1856.csv
/kaggle/input/blend2/blend-2/1855.csv
/kaggle/input/blend1dir/blend/1872.csv
/kaggle/input/blend1dir/blend/1856.csv
/kaggle/input/blend1dir/blend/1855.csv
/kaggle/input/moa-public-kernels/6__submission.csv
/kaggle/input/moa-public-kernels/7__submission.csv
/kaggle/input/moa-public-kernels/1__submission.csv
/kaggle/input/moa-public-kernels/4__submission.csv
/kaggle/input/moa-public-kernels/3__submission.csv
/kaggle/input/moa-public-kernels/kernels.csv
/kaggle/input/moa-public-kernels/5__submission.csv
/kaggle/input/moa-public-kernels/2__submission.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv


In [15]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

public_ids = list(submission['sig_id'].values)

submission.shape

(3982, 207)

In [16]:
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

test_ids = list(test['sig_id'].values)

private_ids = list(set(test_ids)-set(public_ids))

len(private_ids)

0

In [17]:
## use more inferences, at least 2 more

kernels = {
    'kernel':[#'../input/moa-public-kernels/1__submission.csv',
              '../input/moa-public-kernels/2__submission.csv',
              '../input/moa-public-kernels/3__submission.csv',
              '../input/moa-public-kernels/4__submission.csv',
              '../input/moa-public-kernels/5__submission.csv',
             #'../input/moa-public-kernels/6__submission.csv',
             #'../input/blend2/blend-2/1855.csv',
             '../input/blend2/blend-2/1856.csv',     
             '../input/blend2/blend-2/1866.csv',
             #'../input/blend2/blend-2/1872.csv'
    ],
    

    'score':[#0.01900,
            0.01867,
            0.01867,
            0.01871,
            0.01863,
            #0.01873, 
            #0.01855,
            0.01856,
            0.01866,
            #0.01872,
             
            ]
}

# 'kernel':['../input/moa-public-kernels/1__submission.csv',
#               '../input/moa-public-kernels/2__submission.csv',
#               '../input/moa-public-kernels/3__submission.csv',
#               '../input/moa-public-kernels/4__submission.csv',
#               '../input/moa-public-kernels/5__submission.csv',
#               '../input/moa-public-kernels/6__submission.csv',
#              #'../input/blend2/blend-2/1855.csv',
#              '../input/blend2/blend-2/1856.csv',     
#              '../input/blend2/blend-2/1866.csv',
#              '../input/blend2/blend-2/1872.csv'],

In [18]:
kernels=pd.DataFrame(kernels)
kernels

,kernel,score
0,../input/moa-public-kernels/2__submission.csv,0.01867
1,../input/moa-public-kernels/3__submission.csv,0.01867
2,../input/moa-public-kernels/4__submission.csv,0.01871
3,../input/moa-public-kernels/5__submission.csv,0.01863
4,../input/blend2/blend-2/1856.csv,0.01856
5,../input/blend2/blend-2/1866.csv,0.01866


In [19]:
## sonu 

filenames= kernels.kernel.values

predictions = [pd.read_csv(f) for f in filenames]

In [20]:
columns = list(submission.columns)
columns.remove('sig_id')

In [21]:
y_pred = pd.DataFrame()
y_pred['sig_id'] = predictions[0]['sig_id']

for column in columns:
    column_data = []
    for i in range(len(predictions)):
        column_data.append(predictions[i][column])
    y_pred[column] = np.mean(column_data, axis=0)

y_pred.shape

(3982, 207)

In [22]:
scores = kernels['score']

sum_scores = sum(scores)

weights = [x / sum_scores for x in scores]
weights

[0.1668453976764969,
 0.1668453976764969,
 0.1672028596961573,
 0.16648793565683648,
 0.16586237712243077,
 0.16675603217158178]

In [23]:
weighted_y_pred = pd.DataFrame()
weighted_y_pred['sig_id'] = predictions[0]['sig_id']

for column in columns:
    column_data = []
    for i in range(len(predictions)):
        column_data.append(predictions[i][column] * weights[i])
    weighted_y_pred[column] = np.sum(column_data, axis=0)

weighted_y_pred.shape

(3982, 207)

In [24]:
y_pred = weighted_y_pred

In [25]:
submission = pd.DataFrame(index = public_ids + private_ids, columns=columns)
submission.index.name = 'sig_id'

submission[:] = 0

submission.loc[y_pred.sig_id,:] = y_pred[columns].values

submission.loc[test[test.cp_type == 'ctl_vehicle'].sig_id] = 0

submission.to_csv('submission.csv', index=True)

In [26]:
submission.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,0.000885,0.001043,0.001879,0.016856,0.019081,0.004318,0.002055,0.005734,0.000269,0.012792,...,0.000805,0.001667,0.003489,0.001434,0.000602,0.000585,0.000805,0.002010,0.005876,0.001571
id_001897cda,0.000399,0.000994,0.001417,0.002240,0.002015,0.002447,0.003278,0.010371,0.008238,0.012779,...,0.000638,0.001096,0.003113,0.000400,0.011043,0.000552,0.008084,0.000832,0.002799,0.002867
id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
id_00276f245,0.000692,0.000781,0.001892,0.011554,0.010842,0.004273,0.002754,0.004265,0.000491,0.019243,...,0.000599,0.001201,0.003136,0.019173,0.008471,0.000576,0.001719,0.001922,0.001426,0.002935
id_0027f1083,0.001581,0.001429,0.001806,0.015624,0.021616,0.003569,0.005565,0.002199,0.000433,0.011266,...,0.000666,0.000563,0.004185,0.002074,0.000917,0.000661,0.001474,0.001842,0.000325,0.001482
